In [2]:
# from open ai cookbook:
# https://cookbook.openai.com/examples/assistants_api_overview_python

# inits
from openai import OpenAI
from dotenv import load_dotenv
import os
import json

load_dotenv()
api_key = os.getenv("OPENAI_API_KEY")
client = OpenAI(api_key=api_key)

def show_json(obj):
    display(json.loads(obj.model_dump_json()))

In [ ]:
#step 1 - create an assistant with file search enabled
assistant = client.beta.assistants.create(
    name = "Machine learning researcher",
    instructions = "You are a machine learning researcher. Answer questions using the research paper.",
    tools = [{"type": "file_search"}],
    model = "gpt-4o"
)

In [3]:
# Step 2: Upload files and add them to a Vector Store
# https://platform.openai.com/docs/assistants/tools/file-search/step-2-upload-files-and-add-them-to-a-vector-store

vector_store = client.beta.vector_stores.create(name="memgpt_research")
 
# Ready the files for upload to OpenAI
file_paths = ["memgpt-paper.pdf"]
file_streams = [open(path, "rb") for path in file_paths]
 
# Use the upload and poll SDK helper to upload the files, add them to the vector store,
# and poll the status of the file batch for completion.
file_batch = client.beta.vector_stores.file_batches.upload_and_poll(
  vector_store_id=vector_store.id, files=file_streams
)
 
# You can print the status and the file counts of the batch to see the result of this operation.
print(file_batch.status)
print(file_batch.file_counts)

# file = client.files.create(
#     file=open("memgpt-paper.pdf", "rb"),
#     purpose = "assistants"
# )
# print(file)

completed
FileCounts(cancelled=0, completed=1, failed=0, in_progress=0, total=1)


In [ ]:
# step 3: Update the assistant to use the new Vector Store
# To make the files accessible to your assistant, update the assistant’s tool_resources with the new vector_store id.

assistant = client.beta.assistants.update(
  assistant_id=assistant.id,
  tool_resources={"file_search": {"vector_store_ids": [vector_store.id]}},
)

In [107]:
# step 4 - create a thread

# Upload the user provided file to OpenAI
message_file = client.files.create(
  file=open("memgpt-paper.pdf", "rb"), purpose="assistants"
)

# Create a thread and attach the file to the message
thread = client.beta.threads.create(
  messages=[
    {
      "role": "user",
      "content": "Summarize the research paper",
      # Attach the new file to the message.
      "attachments": [
        { "file_id": message_file.id, "tools": [{"type": "file_search"}] }
      ],
    }
  ]
)

print(thread.tool_resources.file_search)

# thread = client.beta.threads.create()
# print(thread)

Thread(id='thread_pasJD1F2G6LV3t4mCNTJUd9j', created_at=1719962089, metadata={}, object='thread', tool_resources=ToolResources(code_interpreter=None, file_search=None))


In [ ]:
# step 5: Crate a run and check the output (without streaming)

# Use the create and poll SDK helper to create a run and poll the status of
# the run until it's in a terminal state.

run = client.beta.threads.runs.create_and_poll(
    thread_id=thread.id, assistant_id=assistant.id
)

messages = list(client.beta.threads.messages.list(thread_id=thread.id, run_id=run.id))

message_content = messages[0].content[0].text
annotations = message_content.annotations
citations = []
for index, annotation in enumerate(annotations):
    message_content.value = message_content.value.replace(annotation.text, f"[{index}]")
    if file_citation := getattr(annotation, "file_citation", None):
        cited_file = client.files.retrieve(file_citation.file_id)
        citations.append(f"[{index}] {cited_file.filename}")

print(message_content.value)
print("\n".join(citations))

In [108]:
# #step 3 - add a message to the thread
# message = client.beta.threads.messages.create(
#     thread_id = thread.id,
#     role = "user",
#     content = "Who are the authors of the paper?"
# )
# print(message)

Message(id='msg_kvEE0nScP7HTprfukeRc7VLy', assistant_id=None, attachments=[], completed_at=None, content=[TextContentBlock(text=Text(annotations=[], value='Who are the authors of the paper?'), type='text')], created_at=1719962106, incomplete_at=None, incomplete_details=None, metadata={}, object='thread.message', role='user', run_id=None, status=None, thread_id='thread_pasJD1F2G6LV3t4mCNTJUd9j')


In [109]:
#step 4 - run the assistant
run = client.beta.threads.runs.create(
    thread_id = thread.id,
    assistant_id = assistant.id
)
print(run)

Run(id='run_JQYNDRavMVbxueKsxuN2m3Jx', assistant_id='asst_7MBFhRnx2cqDuvXWygARr8Ip', cancelled_at=None, completed_at=None, created_at=1719962111, expires_at=1719962711, failed_at=None, incomplete_details=None, instructions='You are a machine learning researcher. Answer questions using the research paper.', last_error=None, max_completion_tokens=None, max_prompt_tokens=None, metadata={}, model='gpt-4o', object='thread.run', parallel_tool_calls=True, required_action=None, response_format='auto', started_at=None, status='queued', thread_id='thread_pasJD1F2G6LV3t4mCNTJUd9j', tool_choice='auto', tools=[FileSearchTool(type='file_search', file_search=None)], truncation_strategy=TruncationStrategy(type='auto', last_messages=None), usage=None, temperature=1.0, top_p=1.0, tool_resources={})


In [110]:
#step 5 - display the assistant's response
run = client.beta.threads.runs.retrieve(
    thread_id = thread.id, 
    run_id = run.id
)
print(run)

Run(id='run_JQYNDRavMVbxueKsxuN2m3Jx', assistant_id='asst_7MBFhRnx2cqDuvXWygARr8Ip', cancelled_at=None, completed_at=1719962113, created_at=1719962111, expires_at=None, failed_at=None, incomplete_details=None, instructions='You are a machine learning researcher. Answer questions using the research paper.', last_error=None, max_completion_tokens=None, max_prompt_tokens=None, metadata={}, model='gpt-4o', object='thread.run', parallel_tool_calls=True, required_action=None, response_format='auto', started_at=1719962111, status='completed', thread_id='thread_pasJD1F2G6LV3t4mCNTJUd9j', tool_choice='auto', tools=[FileSearchTool(type='file_search', file_search=None)], truncation_strategy=TruncationStrategy(type='auto', last_messages=None), usage=Usage(completion_tokens=33, prompt_tokens=1245, total_tokens=1278), temperature=1.0, top_p=1.0, tool_resources={})


In [111]:
messages = client.beta.threads.messages.list(
    thread_id = thread.id
)
print(messages)

SyncCursorPage[Message](data=[Message(id='msg_ezrxxkFVHA86UNaefw1wyvFx', assistant_id='asst_7MBFhRnx2cqDuvXWygARr8Ip', attachments=[], completed_at=None, content=[TextContentBlock(text=Text(annotations=[], value="Please upload the research paper, and I'll be happy to help you find the information you need."), type='text')], created_at=1719962113, incomplete_at=None, incomplete_details=None, metadata={}, object='thread.message', role='assistant', run_id='run_JQYNDRavMVbxueKsxuN2m3Jx', status=None, thread_id='thread_pasJD1F2G6LV3t4mCNTJUd9j'), Message(id='msg_kvEE0nScP7HTprfukeRc7VLy', assistant_id=None, attachments=[], completed_at=None, content=[TextContentBlock(text=Text(annotations=[], value='Who are the authors of the paper?'), type='text')], created_at=1719962106, incomplete_at=None, incomplete_details=None, metadata={}, object='thread.message', role='user', run_id=None, status=None, thread_id='thread_pasJD1F2G6LV3t4mCNTJUd9j')], object='list', first_id='msg_ezrxxkFVHA86UNaefw1wyvF

In [112]:
for message in reversed(messages.data): 
    print(message.role + ": " + message.content[0].text.value)

user: Who are the authors of the paper?
assistant: Please upload the research paper, and I'll be happy to help you find the information you need.
